
# 03_ai_agent_analysis_part3_fixed — Config機能追加 & Controller API（改修版）
- セル1: `load_configuration()`（Config読み込み・検証・マージ）
- セル2: Robust handoff loader（Part2受け取り）
- セル3: DB設定管理（cfg/handoff/env優先順位）
- セル4: Handoff展開（必須キー検証）
- セル5: **TLD分布分析**（中核機能を保持したままConfig反映）
- セル6: Handoff保存（pkl + JSON）
- セル7: Controller API `explanation_quality()`（エイリアス `tld_risk_analysis`）


In [1]:
# === Cell 0 (02以降 共通): レジストリから解決して paths を読む ===
import run_id_registry as runreg
rid = runreg.bootstrap()  # env→ファイル(artifacts/_current/run_id.txt)→Part3→latest→新規 の順で解決

import importlib
import _compat.paths as paths
importlib.reload(paths)
importlib.reload(paths)
print("[NX] RUN_ID =", rid, "| paths.RUN_ID =", paths.RUN_ID)


[NX] RUN_ID = 2026-01-10_140940 | paths.RUN_ID = 2026-01-10_140940


In [2]:

# === セル1: 環境初期化・パス設定 + load_configuration ===
import os, json, typing, warnings
from pathlib import Path
from datetime import datetime
try:
    import yaml
except Exception:
    yaml = None

if 'RUN_ID' not in globals():
    RUN_ID = os.environ.get("RUN_ID") or datetime.now().strftime("%Y-%m-%d_%H%M%S")
ARTIFACTS = Path("artifacts") / RUN_ID
for _p in [ARTIFACTS / d for d in ["raw","processed","models","results","handoff","logs","traces"]]:
    _p.mkdir(parents=True, exist_ok=True)

RAW, PROCESSED, MODELS, RESULTS, HANDOFF, LOGS, TRACES = [ARTIFACTS / d for d in ["raw","processed","models","results","handoff","logs","traces"]]
RAW_DIR, PROCESSED_DIR, MODELS_DIR, RESULTS_DIR, HANDOFF_DIR, LOGS_DIR, TRACES_DIR = map(str, [RAW, PROCESSED, MODELS, RESULTS, HANDOFF, LOGS, TRACES])
print(f"✅ IO guard ready -> artifacts/{RUN_ID}")

def _deep_update(base: dict, override: dict) -> dict:
    base = dict(base or {})
    for k, v in (override or {}).items():
        if isinstance(v, dict) and isinstance(base.get(k), dict):
            base[k] = _deep_update(base[k], v)
        else:
            base[k] = v
    return base

from typing import Optional, Dict, Any
def load_configuration(config_path: Optional[str] = None,
                       cfg_override: Optional[Dict[str, Any]] = None) -> Dict[str, Any]:
    defaults = {
        "system": {"cert_only_mode": True, "seed": 42},
        "db": {"dbname":"rapids_data","user":"postgres","password":"asomura","host":"localhost","port":"5432","timeout_s":30,"read_only":True},
        "tld_analysis": {"enabled":True,"use_all_data":True,"balance_data":True,"sample_size":None,"min_sample_size":10,
                         "max_dangerous_tlds":30,"max_legitimate_tlds":30,"percentile_thresholds":[50,75,90],"fn_weight":1.5},
        "engine": {"max_concurrent":20,"max_retries":3,"batch_size":10000},
        "paths": {"base_dir":"artifacts"},
        "llm": {"enabled":False}
    }
    cfg = dict(defaults)
    if config_path and Path(config_path).exists():
        try:
            text = Path(config_path).read_text(encoding="utf-8")
            if Path(config_path).suffix.lower() in (".yml",".yaml") and yaml:
                cfg = _deep_update(cfg, yaml.safe_load(text) or {})
            else:
                cfg = _deep_update(cfg, json.loads(text))
        except Exception as e:
            warnings.warn(f"設定ファイル読込失敗: {e}")
    env_map = {"USE_ALL_DATA":("tld_analysis","use_all_data"),
               "BALANCE_DATA":("tld_analysis","balance_data"),
               "PGDATABASE":("db","dbname"),
               "PGUSER":("db","user"),
               "PGPASSWORD":("db","password")}
    for env, path in env_map.items():
        if env in os.environ and os.environ[env]!="":
            val = os.environ[env]
            if isinstance(val,str) and val.lower() in ("true","false"):
                val = (val.lower()=="true")
            node = cfg
            for k in path[:-1]:
                node = node.setdefault(k,{})
            node[path[-1]] = val
    if isinstance(cfg_override, dict):
        cfg = _deep_update(cfg, cfg_override)
    if not cfg["system"].get("cert_only_mode", False):
        raise ValueError("system.cert_only_mode は True 必須です")
    if not cfg.get("tld_analysis",{}).get("enabled", False):
        raise ValueError("tld_analysis.enabled は True 必須です")
    for k in ("dbname","user","host","port"):
        if not str(cfg["db"].get(k,"")).strip():
            raise ValueError(f"DB必須項目不足: db.{k}")
    globals()["cfg"] = cfg
    print("🔧 Config loaded (cert_only_mode=True, tld_analysis.enabled=True)")
    return cfg

cfg = load_configuration(os.getenv("CONFIG_PATH"), None)


✅ IO guard ready -> artifacts/2026-01-10_140940
🔧 Config loaded (cert_only_mode=True, tld_analysis.enabled=True)


In [3]:

# === セル2: Robust handoff loader ===
import joblib, glob
Path(HANDOFF_DIR).mkdir(parents=True, exist_ok=True)
cands = [str(Path(HANDOFF_DIR) / "03_ai_agent_analysis_part2.pkl")]
if not any(Path(p).exists() for p in cands):
    cands += sorted(glob.glob("artifacts/*/handoff/03_ai_agent_analysis_part2.pkl"))
src = next((p for p in cands if Path(p).exists()), None)
if not src:
    raise FileNotFoundError("Part2 handoff が見つかりません。")
handoff = joblib.load(src)
print(f"✅ handoff loaded: {src}")


✅ handoff loaded: artifacts/2026-01-10_140940/handoff/03_ai_agent_analysis_part2.pkl


In [4]:

# === セル3: DB設定管理 ===
import os
def _db_from_env():
    return {'dbname':os.getenv('PGDATABASE','rapids_data'),
            'user':os.getenv('PGUSER','postgres'),
            'password':os.getenv('PGPASSWORD','asomura'),
            'host':os.getenv('PGHOST','localhost'),
            'port':os.getenv('PGPORT','5432')}
def build_db_config(cfg: dict, handoff: dict) -> dict:
    if isinstance(cfg, dict) and 'db' in cfg:
        base = {k:str(cfg['db'].get(k)) for k in ['dbname','user','password','host','port']}
    elif isinstance(handoff, dict) and 'DB_CONFIG' in handoff:
        base = dict(handoff['DB_CONFIG'])
    else:
        base = _db_from_env()
    base['connect_timeout'] = int((cfg.get('db',{}) or {}).get('timeout_s',30)) if isinstance(cfg,dict) else 30
    base['_read_only'] = bool((cfg.get('db',{}) or {}).get('read_only',True)) if isinstance(cfg,dict) else True
    return base
DB_CONFIG = build_db_config(cfg, handoff)
print("🔧 DB_CONFIG ready:", {k: DB_CONFIG[k] for k in ['dbname','host','port','user']}, f"(timeout={DB_CONFIG['connect_timeout']}s)")


🔧 DB_CONFIG ready: {'dbname': 'rapids_data', 'host': 'localhost', 'port': '5432', 'user': 'postgres'} (timeout=30s)


In [5]:

# === セル4: Handoff展開 ===
required = ['false_negatives_df','brand_keywords','cert_full_info_map','fn_features_df']
miss = [k for k in required if k not in handoff]
if miss: raise KeyError(f"handoff keys missing: {miss}")
false_negatives_df = handoff['false_negatives_df']
brand_keywords = handoff['brand_keywords']
cert_full_info_map = handoff['cert_full_info_map']
fn_features_df = handoff['fn_features_df']
if 'DB_CONFIG' in handoff: DB_CONFIG = {**DB_CONFIG, **handoff['DB_CONFIG']}
print(f"✅ unpack: fn_rows={getattr(false_negatives_df,'shape',[None,None])[0]}, brand_keywords={len(brand_keywords) if isinstance(brand_keywords,(list,dict,set,tuple)) else 'n/a'}")


✅ unpack: fn_rows=55524, brand_keywords=109


In [6]:

# === セル5: TLD分布分析（中核保持 + Config） ===
import psycopg2
from psycopg2.extras import RealDictCursor
from collections import Counter
from urllib.parse import urlparse
import numpy as np, random

ta = cfg.get('tld_analysis', {})
use_all_data   = bool(ta.get('use_all_data', True))
balance_data   = bool(ta.get('balance_data', True))
sample_size    = ta.get('sample_size', None)
min_sample     = int(ta.get('min_sample_size', 10))
max_dangerous  = int(ta.get('max_dangerous_tlds', 30))
max_legitimate = int(ta.get('max_legitimate_tlds', 30))
pct_thresholds = list(ta.get('percentile_thresholds', [50,75,90]))
fn_weight      = float(ta.get('fn_weight', 1.5))
random.seed(cfg.get('system',{}).get('seed',42))

def extract_tld(domain):
    if not domain: return None
    if '://' in domain:
        try: domain = urlparse(domain).netloc
        except Exception: pass
    domain = domain.split(':')[0]
    parts = domain.split('.')
    if len(parts) >= 2:
        if len(parts) >= 3 and parts[-2] in ['co','ac','or','ne','go']:
            return f'.{parts[-2]}.{parts[-1]}'
        return f'.{parts[-1]}'
    return None

conn = psycopg2.connect(dbname=DB_CONFIG['dbname'], user=DB_CONFIG['user'], password=DB_CONFIG['password'],
                        host=DB_CONFIG['host'], port=DB_CONFIG['port'],
                        connect_timeout=int(DB_CONFIG.get('connect_timeout',30)))
cur = conn.cursor(cursor_factory=RealDictCursor)
print("✅ DB connected")

phishing_queries = {
    'phishtank': "SELECT cert_domain as domain FROM phishtank_entries WHERE cert_status='SUCCESS' AND cert_data IS NOT NULL AND cert_domain IS NOT NULL",
    'jpcert':    "SELECT domain FROM jpcert_phishing_urls WHERE status='SUCCESS' AND cert_data IS NOT NULL AND domain IS NOT NULL",
    'certificates': "SELECT domain FROM certificates WHERE status='SUCCESS' AND cert_data IS NOT NULL AND domain IS NOT NULL"
}
ph_dom = []; ph_tlds = Counter()
for name, q in phishing_queries.items():
    if not use_all_data and sample_size: q += f" LIMIT {int(sample_size)}"
    cur.execute(q); rows = cur.fetchall()
    st = Counter()
    for r in rows:
        d = r['domain']
        if d:
            ph_dom.append(d)
            t = extract_tld(d)
            if t:
                st[t]+=1; ph_tlds[t]+=1
    print(f"  {name}: {len(rows):,}件 (Top5: {', '.join([f'{t}({c})' for t,c in st.most_common(5)])})")

q = "SELECT domain FROM trusted_certificates WHERE status='SUCCESS' AND cert_data IS NOT NULL AND domain IS NOT NULL"
if not use_all_data and sample_size: q += f" LIMIT {int(sample_size)}"
cur.execute(q); rows = cur.fetchall()
tr_dom = []; tr_tlds = Counter()
for r in rows:
    d = r['domain']
    if d:
        tr_dom.append(d)
        t = extract_tld(d)
        if t: tr_tlds[t]+=1

if balance_data:
    minc = min(len(set(ph_dom)), len(set(tr_dom)))
    ph_u = list(set(ph_dom)); tr_u = list(set(tr_dom))
    random.seed(cfg.get('system',{}).get('seed',42))
    ph_bal = random.sample(ph_u, minc) if len(ph_u)>minc else ph_u
    tr_bal = random.sample(tr_u, minc) if len(tr_u)>minc else tr_u
else:
    ph_bal, tr_bal = ph_dom, tr_dom

ph_bal_t = Counter(extract_tld(d) for d in ph_bal if extract_tld(d))
tr_bal_t = Counter(extract_tld(d) for d in tr_bal if extract_tld(d))

from collections import Counter as _Ctr
fn_tlds = _Ctr()
if hasattr(false_negatives_df,'columns') and 'domain' in false_negatives_df.columns:
    for d in false_negatives_df['domain']:
        t = extract_tld(d)
        if t: fn_tlds[t]+=1

dangerous_ratio = {}
for t in ph_bal_t:
    ph = ph_bal_t[t]; tr = tr_bal_t.get(t,0)
    if ph >= min_sample:
        ratio = float('inf') if tr==0 else ph/(tr+1)
        dangerous_ratio[t] = {'ratio':ratio,'phishing_count':ph,'trusted_count':tr,
                              'phishing_pct': ph/max(len(ph_bal),1)*100}

ratios = [s['ratio'] for s in dangerous_ratio.values() if s['ratio']!=float('inf')]
if ratios:
    pcts = np.percentile(ratios, pct_thresholds).tolist()
else:
    pcts = [10,10,10]
p50,p75,p90 = (pcts+[10,10,10])[:3]

DANGEROUS_TLDS = []
for t,s in sorted(dangerous_ratio.items(), key=lambda x:x[1]['ratio'], reverse=True):
    if ((s['ratio']==float('inf') and s['phishing_count']>=10) or
        (s['ratio']>=p90 and s['phishing_pct']>=0.1) or
        (s['ratio']>=10 and s['phishing_pct']>=1.0)):
        DANGEROUS_TLDS.append(t)
        if len(DANGEROUS_TLDS)>=max_dangerous: break

LEGITIMATE_TLDS = []
for t,c in tr_bal_t.most_common(max_legitimate*2):
    tr_pct = c/max(len(tr_bal),1)*100
    if tr_pct>=1.0 and c>=1000:
        ph = ph_bal_t.get(t,0)
        ratio = ph/c if c>0 else 0.0
        if ratio<0.5:
            LEGITIMATE_TLDS.append(t)
    if len(LEGITIMATE_TLDS)>=max_legitimate: break

if len(fn_tlds):
    for t,c in fn_tlds.most_common():
        fn_pct = c/max(len(false_negatives_df),1)*100
        if fn_pct>=1.0 and t not in DANGEROUS_TLDS and t not in LEGITIMATE_TLDS:
            if t in dangerous_ratio and dangerous_ratio[t]['ratio']>=5:
                DANGEROUS_TLDS.append(t)

NEUTRAL_TLDS = []
for t in ['.com','.org','.net','.info','.biz']:
    if t not in DANGEROUS_TLDS and t not in LEGITIMATE_TLDS:
        if t in ph_bal_t and t in tr_bal_t:
            ph_pct = ph_bal_t[t]/max(len(ph_bal),1)*100
            tr_pct = tr_bal_t[t]/max(len(tr_bal),1)*100
            if ph_pct>=0.5 or tr_pct>=0.5:
                NEUTRAL_TLDS.append(t)

globals().update({'DANGEROUS_TLDS':DANGEROUS_TLDS,'LEGITIMATE_TLDS':LEGITIMATE_TLDS,'NEUTRAL_TLDS':NEUTRAL_TLDS,
                  'phishing_tld_stats':ph_bal_t,'trusted_tld_stats':tr_bal_t})
cur.close(); conn.close()
print(f"✅ TLD分析完了: dangerous={len(DANGEROUS_TLDS)}, legitimate={len(LEGITIMATE_TLDS)}, neutral={len(NEUTRAL_TLDS)}")


✅ DB connected
  phishtank: 54,362件 (Top5: .com(23196), .dev(5865), .ly(3191), .me(2732), .de(2705))
  jpcert: 116,647件 (Top5: .com(35795), .cn(34513), .top(6972), .org(5610), .shop(4572))
  certificates: 196,392件 (Top5: .org(102083), .com(30237), .cn(27498), .top(7450), .xyz(6099))
✅ TLD分析完了: dangerous=23, legitimate=8, neutral=1


In [7]:

# === セル6: Handoff保存（pkl + JSON） ===
import joblib, json, time
from pathlib import Path
out_handoff = Path(HANDOFF_DIR) / "03_ai_agent_analysis_part3.pkl"
payload = {
    'false_negatives_df': false_negatives_df,
    'brand_keywords': brand_keywords,
    'cert_full_info_map': cert_full_info_map,
    'fn_features_df': fn_features_df,
    'DANGEROUS_TLDS': DANGEROUS_TLDS,
    'LEGITIMATE_TLDS': LEGITIMATE_TLDS,
    'NEUTRAL_TLDS': NEUTRAL_TLDS,
    'phishing_tld_stats': phishing_tld_stats,
    'trusted_tld_stats': trusted_tld_stats,
    'DB_CONFIG': DB_CONFIG,
    'cfg': cfg
}
joblib.dump(payload, str(out_handoff))

Path(RESULTS_DIR).mkdir(parents=True, exist_ok=True)
with open(Path(RESULTS_DIR)/"tld_statistics.json", "w", encoding="utf-8") as f:
    json.dump({'ts': time.time(),
               'dangerous_tlds': list(DANGEROUS_TLDS),
               'legitimate_tlds': list(LEGITIMATE_TLDS),
               'neutral_tlds': list(NEUTRAL_TLDS)}, f, ensure_ascii=False, indent=2)
print(f"💾 Saved handoff: {out_handoff}")
print(f"💾 Saved JSON   : {Path(RESULTS_DIR)/'tld_statistics.json'}")


💾 Saved handoff: artifacts/2026-01-10_140940/handoff/03_ai_agent_analysis_part3.pkl
💾 Saved JSON   : artifacts/2026-01-10_140940/results/tld_statistics.json


In [8]:

# === セル7: Controller API関数 ===
from typing import Tuple, Dict, Any
import traceback
from pathlib import Path

def explanation_quality(session_id: str, cfg: Dict[str, Any]) -> Tuple[str, Dict[str, str]]:
    try:
        if not isinstance(cfg, dict):
            return "INVALID_INPUT", {"error":"cfgは辞書型で指定してください"}
        _ = load_configuration(cfg_override=cfg)
        if not _.get('tld_analysis',{}).get('enabled', False):
            return "INVALID_INPUT", {"error":"tld_analysis.enabled が無効"}
        if not Path(HANDOFF_DIR, "03_ai_agent_analysis_part2.pkl").exists():
            return "NOT_FOUND", {"error":"Part2 handoff が見つかりません"}
        paths = {
            "dangerous_tlds": str(Path(RESULTS_DIR)/"tld_statistics.json"),
            "legitimate_tlds": str(Path(RESULTS_DIR)/"tld_statistics.json"),
            "neutral_tlds": str(Path(RESULTS_DIR)/"tld_statistics.json"),
            "tld_statistics": str(Path(RESULTS_DIR)/"tld_statistics.json"),
            "handoff": str(Path(HANDOFF_DIR)/"03_ai_agent_analysis_part3.pkl"),
            "logs": LOGS_DIR
        }
        if len(DANGEROUS_TLDS)==0 and len(LEGITIMATE_TLDS)==0:
            return "NOT_FOUND", {"error":"TLDリストが空です", **paths}
        return "OK", paths
    except Exception as e:
        return "ERROR", {"error": str(e), "trace": traceback.format_exc()}

tld_risk_analysis = explanation_quality
print("✅ Controller API ready: explanation_quality() / tld_risk_analysis()")


✅ Controller API ready: explanation_quality() / tld_risk_analysis()
